In [8]:
!pip install requests pandas scikit-learn

In [9]:
# Install required libraries (if not already installed)
!pip install requests numpy tensorflow 

import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
import os
!pip install folium


In [12]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np
import requests
import os
import folium
from geopy.geocoders import Nominatim
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [14]:


def load_influenza_data(file_path="influenza_data.csv"):
    try:
        df = pd.read_csv(file_path)
        required_columns = {"temperature_c", "humidity_pct", "pressure_hpa", "rainfall_mm", "age", "symptom_days", "influenza_positive"}
        
        if not required_columns.issubset(df.columns):
            raise ValueError("Missing required columns in influenza dataset!")
        
        return df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

def get_weather_data(city, api_key=None):
    if not api_key:
        return {
            "temperature_c": 22.0,
            "humidity_pct": 70,
            "pressure_hpa": 1012,
            "rainfall_mm": 5.0
        }
    
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url).json()
        return {
            "temperature_c": response["main"]["temp"],
            "humidity_pct": response["main"]["humidity"],
            "pressure_hpa": response["main"]["pressure"],
            "rainfall_mm": response.get("rain", {}).get("1h", 0)
        }
    except:
        return get_weather_data(city, None)

def get_country_coordinates(country):
    """Gets latitude and longitude for a given country."""
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(country)
    return (location.latitude, location.longitude) if location else (None, None)

def plot_disease_risk_on_map(country, risk_percentage):
    """Plots influenza outbreak risk on a map."""
    lat, lon = get_country_coordinates(country)
    if lat is None or lon is None:
        print(f"Could not find coordinates for {country}")
        return
    
    world_map = folium.Map(location=[lat, lon], zoom_start=3)
    folium.Marker(
        [lat, lon], 
        popup=f"{country}: {risk_percentage:.2f}% Risk", 
        icon=folium.Icon(color="red")
    ).add_to(world_map)
    world_map.save("influenza_risk_map.html")
    print("Map saved as influenza_risk_map.html")

def train_model():
    df = load_influenza_data()
    if df is None:
        return None

    X = df.drop(columns=["influenza_positive"])
    y = df["influenza_positive"]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train)

    return model, scaler

def predict_influenza(country="USA"):
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")
    weather = get_weather_data(country, api_key)

    model, scaler = train_model()
    if model is None:
        return "Model training failed."

    # Prepare new data sample
    X_new = pd.DataFrame([{
        "temperature_c": weather["temperature_c"],
        "humidity_pct": weather["humidity_pct"],
        "pressure_hpa": weather["pressure_hpa"],
        "rainfall_mm": weather["rainfall_mm"],
        "age": 35,  
        "symptom_days": 2 
    }])

    X_new_scaled = scaler.transform(X_new)
    risk_prediction = model.predict_proba(X_new_scaled)[0][1] * 100  

    plot_disease_risk_on_map(country, risk_prediction)
    return f"Influenza outbreak risk in {country}: {risk_prediction:.2f}%"



In [17]:
model, _ = train_model()
model.save_model("influenza_model.h5")

c:\Users\siddh\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:11:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\siddh\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:11:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
